# VacationPy
### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df=pd.read_csv('../WeatherPy/cities_df.csv')
vacation_df

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,0,georgetown,MY,5.4112,100.3354,76.89,92,20,2.30
1,1,willmar,US,45.1219,-95.0433,80.19,29,40,13.80
2,2,avarua,CK,-21.2078,-159.7750,75.25,94,75,13.80
3,3,tuatapere,NZ,-46.1333,167.6833,41.83,91,100,4.21
4,4,bobcaygeon,CA,44.5462,-78.5420,63.63,91,100,7.20
...,...,...,...,...,...,...,...,...,...
619,619,lima,PE,-12.0432,-77.0282,67.89,75,90,8.05
620,620,nadterechnoye,RU,43.6227,45.3397,76.08,57,98,2.35
621,621,alyangula,AU,-13.8483,136.4192,64.27,100,90,4.61
622,622,mitsamiouli,KM,-11.3847,43.2844,76.44,71,0,3.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#url=f"krrps://maps.gooleapis.com/maps.api

# Store 'Lat' and 'Lng' into  locations and humidity.
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
humidity = vacation_df['Humidity'].astype(float)
maxhumidity = humidity.max()

# Make the base map.
fig = gmaps.figure()
#Add the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius = 2)

fig.add_layer(heat_layer)
#print the figure.
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
vacation_thoughts_df = vacation_df.loc[(vacation_df['Temperature']>65)&(vacation_df['Temperature']<80)&(vacation_df['Wind']<10)&(vacation_df['Humidity']<35)&(vacation_df['Cloudiness']<20),:]
vacation_thoughts_df.reset_index(drop=True)


,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind
0,125,catamarca,AR,-28.4696,-65.7852,70.77,34,0,4.00
1,351,malanje,AO,-9.5402,16.3410,69.44,29,8,3.22
2,369,campos belos,BR,-13.0367,-46.7717,78.55,33,6,4.74


## Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=pd.DataFrame(vacation_thoughts_df, columns=['City','Country','Latitude','Longitude','Humidity','Cloudiness','Wind'])

hotel_df['Hotel Name']=""
hotel_df

,City,Country,Latitude,Longitude,Humidity,Cloudiness,Wind,Hotel Name
125,catamarca,AR,-28.4696,-65.7852,34,0,4.00,
351,malanje,AO,-9.5402,16.3410,29,8,3.22,
369,campos belos,BR,-13.0367,-46.7717,33,6,4.74,


In [6]:
hotels = []

for i in range(len(hotel_df)):
    lat=hotel_df.iloc[i]['Latitude']
    lng=hotel_df.iloc[i]['Longitude']
    
    params = {
        "location":f'{lat},{lng}',
        "radius":5000,
        "types":"hotel",
        "key":g_key
    }
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response =  requests.get(url,params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
    except:
        hotels.append("")
hotel_df["Hotel Name"]=hotels
hotel_df

,City,Country,Latitude,Longitude,Humidity,Cloudiness,Wind,Hotel Name
125,catamarca,AR,-28.4696,-65.7852,34,0,4.00,Catamarca
351,malanje,AO,-9.5402,16.3410,29,8,3.22,Malanje
369,campos belos,BR,-13.0367,-46.7717,33,6,4.74,Campos Belos


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Create Marker Layer
locations = hotel_df[["Latitude", "Longitude"]]
hotel_layer = gmaps.marker_layer(
     locations,                                                       
     info_box_content= hotel_info
)

# fill_color='red',
#      stroke_color='rgba(0, 0, 150, 0.4)', scale=8, 
fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig                      

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))